In [2]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from splinter import Browser
from io import StringIO
import time
import requests
import datetime as dt
import openpyxl
from openpyxl import workbook
from openpyxl import load_workbook

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common import keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import warnings
warnings.filterwarnings("ignore")

In [3]:
#install new Chrome webdriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Will_Boisseau\.wdm\drivers\chromedriver\win32\101.0.4951.41]


In [9]:
# Scrape Morningstar

# Create webdriver
driver=webdriver.Chrome()
driver.get("https://www.morningstar.com/etfs/xnas/acwi/portfolio")
#print(value)

# Close the browser after scraping
#driver.quit()


In [10]:
time.sleep(2)

In [11]:
#full xpath: /html/body/div[2]/div/div/div[2]/div[3]/div/main/div[2]/div/div/div[1]/div[2]/sal-components/div/sal-components-funds-portfolio/div/div[1]/div[2]/div/div[6]/div[1]/div/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[2]/div[1]/table/tbody/tr

rows = 1 +len(driver.find_elements_by_xpath('/html/body/div[2]/div/div/div[2]/div[3]/div/main/div[2]/div/div/div[1]/div[2]/sal-components/div/sal-components-funds-portfolio/div/div[1]/div[2]/div/div[6]/div[1]/div/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[2]/div[1]/table/tbody/tr'))
cols=len(driver.find_elements_by_xpath('/html/body/div[2]/div/div/div[2]/div[3]/div/main/div[2]/div/div/div[1]/div[2]/sal-components/div/sal-components-funds-portfolio/div/div[1]/div[2]/div/div[6]/div[1]/div/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[2]/div[1]/table/tbody/tr[1]/td'))
list_1 = []
for i in range(1, rows):
    list_2 = []
    for j in range(1, cols+1):
        value = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[3]/div/main/div[2]/div/div/div[1]/div[2]/sal-components/div/sal-components-funds-portfolio/div/div[1]/div[2]/div/div[6]/div[1]/div/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[2]/div[1]/table/tbody/tr['+str(i)+']/td['+str(j)+']').text
        print(value, end='     ')
        list_2.append(value)
    list_1.append(list_2)
    print()

Basic Materials     5.01     5.40     
Consumer Cyclical     10.13     10.46     
Financial Services     15.99     14.07     
Real Estate     2.90     2.42     
Communication Services     7.97     7.12     
Energy     5.34     4.21     
Industrials     9.48     14.27     
Technology     19.65     18.00     
Consumer Defensive     7.55     7.75     
Healthcare     12.83     12.37     
Utilities     3.16     3.94     


In [12]:
#put the list into a dataframe
df=pd.DataFrame(list_1)
df

,0,1,2
0,Basic Materials,5.01,5.40
1,Consumer Cyclical,10.13,10.46
2,Financial Services,15.99,14.07
3,Real Estate,2.90,2.42
4,Communication Services,7.97,7.12
5,Energy,5.34,4.21
6,Industrials,9.48,14.27
7,Technology,19.65,18.00
8,Consumer Defensive,7.55,7.75
9,Healthcare,12.83,12.37


In [13]:
#drop the last column, rename remaining columns

df2= df.drop([2],axis=1)
df2.rename(columns={0:'GICS Sector',1:'ACWI Weight'},inplace=True)

#rename sectors to match GICS sectors
df2['GICS Sector']=df2['GICS Sector'].replace({'Consumer Cyclical':'Consumer Discretionary','Basic Materials':'Materials','Financial Services':'Financials','Technology':'Information Technology','Consumer Defensive':'Consumer Staples'})
df2



,GICS Sector,ACWI Weight
0,Materials,5.01
1,Consumer Discretionary,10.13
2,Financials,15.99
3,Real Estate,2.90
4,Communication Services,7.97
5,Energy,5.34
6,Industrials,9.48
7,Information Technology,19.65
8,Consumer Staples,7.55
9,Healthcare,12.83


In [15]:
#get latest weightings for each holding

today = dt.datetime.today().strftime("%m.%d.%Y")
ClosePath=f'C:/Users/Will_Boisseau/OneDrive - Georgetown University/Graduate Investment Fund (GIF)/All Things Investing/Performance & Research/Performance/Brokerage Data_for all club members/GIF_historical_closes{today}.xlsx'

weightdf=pd.read_excel(ClosePath,sheet_name='GIF_YTD_Perf',index_col=0)

#count number of columns. Today's market values are at index cols-6
cols=weightdf.shape[1]
cols

#dataframe should just be ticker and current value
weightdf=weightdf.iloc[:,[0,cols-6]]

#calculate weight of each position based on total portfolio MV
MVsum=weightdf['Latest_Total_Value'].sum()
weightdf['GIF_Weight']=weightdf['Latest_Total_Value']/MVsum
weightdf


,Ticker,Latest_Total_Value,GIF_Weight
0,AON,35773.359070,0.044561
1,AAPL,36661.799850,0.045667
2,BRK-B,25528.798828,0.031800
3,CAT,19759.739944,0.024613
4,CI,47942.398926,0.059719
5,DHI,16825.499725,0.020958
6,HON,12566.400391,0.015653
7,RHS,13649.200439,0.017002
8,MU,23609.039902,0.029408
9,NSRGY,17021.200256,0.021202


In [22]:
#get path of cost basis file, which has GICS sectors listed
BasisPath=f'C:/Users/Will_Boisseau/OneDrive - Georgetown University/Graduate Investment Fund (GIF)/All Things Investing/Performance & Research/Performance/Brokerage Data_for all club members/GIF_CostBasis_asof{today}.xlsx'

#get GICS sector from above file. Then dataframe should just be ticker and GICS sector
basisdf=pd.read_excel(BasisPath,sheet_name='GIF_attributes-cost',index_col=0)
basisdf=basisdf.iloc[:,[1,13]]

#combine the frames and match ticker w/ correct sector
combdf=pd.merge(weightdf,basisdf,on="Ticker",how='left')
combdf[['GICS_Sector']]=combdf[['GICS_Sector']].fillna("Cash")
combdf.rename(columns={'GICS_Sector':'GICS Sector','GIF_Weight':'GIF Weight'},inplace=True)

#remove excess whitespace in the GICS Sector fields
combdf['GICS Sector'] = combdf['GICS Sector'].str.strip()
combdf


,Ticker,Latest_Total_Value,GIF Weight,GICS Sector
0,AON,35773.359070,0.044561,Financials
1,AAPL,36661.799850,0.045667,Information Technology
2,BRK-B,25528.798828,0.031800,Financials
3,CAT,19759.739944,0.024613,Industrials
4,CI,47942.398926,0.059719,Healthcare
5,DHI,16825.499725,0.020958,Consumer Discretionary
6,HON,12566.400391,0.015653,Industrials
7,RHS,13649.200439,0.017002,Consumer Staples
8,MU,23609.039902,0.029408,Information Technology
9,NSRGY,17021.200256,0.021202,Consumer Staples


In [23]:
#calculate weight of each sector, drop the total value column

GIFsecdf=combdf.groupby('GICS Sector').sum()
GIFsecdf= GIFsecdf.drop(["Latest_Total_Value"],axis=1)
GIFsecdf


,GIF Weight
GICS Sector,
Cash,0.036698
Communication Services,0.046678
Consumer Discretionary,0.073510
Consumer Staples,0.088136
Energy,0.044926
Financials,0.159989
Healthcare,0.159680
Industrials,0.090240
Information Technology,0.168240


In [24]:
#combine the GIF and ACWI weight dataframes

#combine the frames and match ticker w/ correct sector, use 0 for ACWI Cash weight, cast both weights as float,
#state weights as %, compute active weight as GIF-ACWI

activedf=pd.merge(GIFsecdf,df2,on="GICS Sector",how='left')

activedf['ACWI Weight'] = activedf['ACWI Weight'].fillna(0)
activedf['ACWI Weight'] = activedf['ACWI Weight'].astype(float)
activedf['GIF Weight']=activedf['GIF Weight']*100
activedf['GIF Weight'] = activedf['GIF Weight'].astype(float)
activedf['Active Weight']=activedf['GIF Weight']-activedf['ACWI Weight']
activedf



,GICS Sector,GIF Weight,ACWI Weight,Active Weight
0,Cash,3.669776,0.00,3.669776
1,Communication Services,4.667786,7.97,-3.302214
2,Consumer Discretionary,7.351021,10.13,-2.778979
3,Consumer Staples,8.813634,7.55,1.263634
4,Energy,4.492597,5.34,-0.847403
5,Financials,15.998882,15.99,0.008882
6,Healthcare,15.967970,12.83,3.137970
7,Industrials,9.023957,9.48,-0.456043
8,Information Technology,16.823950,19.65,-2.826050
9,Materials,5.005285,5.01,-0.004715


In [26]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
ActivePath=f"C:/Users/Will_Boisseau/OneDrive - Georgetown University/Graduate Investment Fund (GIF)/All Things Investing/Performance & Research/Performance/Brokerage Data_for all club members/Active Weights/Active_Weights_asof{today}.xlsx"

writer = pd.ExcelWriter(ActivePath, engine='xlsxwriter')
#writer=pd.ExcelWriter(ActivePath,engine=xlsxwriter)
activedf.to_excel(writer,sheet_name='Active Weights')
writer.save()


#GIF_combined_cost['Report_data'] = dt.datetime.today().strftime("%m/%d/%Y")
#today = dt.datetime.today().strftime("%m.%d.%Y")
#writer = pd.ExcelWriter(f"Output_Data/GIF_CostBasis_asof{today}.xlsx", engine='xlsxwriter')


# Write each dataframe to a different worksheet.
#GIF_combined_attrib.to_excel(writer, sheet_name='GIF_attributes-cost')
#GIF_combined_cost.to_excel(writer, sheet_name='cost_raw')

# Close the Pandas Excel writer and output the Excel file.
#writer.save()
